In [1]:
from pathlib import Path

# ---------------------------------------------------------
# 경로 설정
# ---------------------------------------------------------
BASE_DIR = Path().resolve().parent.parent.parent
base_path =  BASE_DIR
tn_base_path = base_path / "data" / "train" / "TL_csv"
tn_visit_area_info_H_path = tn_base_path / "tn_visit_area_info_방문지정보_H.csv"
vn_base_path = base_path / "data" / "valid" / "VL_csv"
vn_visit_area_info_H_path = vn_base_path / "tn_visit_area_info_방문지정보_H.csv"

save_path = base_path / "AI 모델" / "1.모델소스코드" / "1.여행로그 장소 추천 고도화" / "data"
print(tn_visit_area_info_H_path)
print(vn_visit_area_info_H_path)
print(save_path)


C:\Users\User\Desktop\xodls\through_pjt_ai\data\train\TL_csv\tn_visit_area_info_방문지정보_H.csv
C:\Users\User\Desktop\xodls\through_pjt_ai\data\valid\VL_csv\tn_visit_area_info_방문지정보_H.csv
C:\Users\User\Desktop\xodls\through_pjt_ai\AI 모델\1.모델소스코드\1.여행로그 장소 추천 고도화\data


In [2]:
import pandas as pd
import numpy as np
# ---------------------------------------------------------
# 데이터 Load, Concat
# ---------------------------------------------------------

train_visit_H = pd.read_csv(tn_visit_area_info_H_path)
valid_visit_H = pd.read_csv(vn_visit_area_info_H_path)
visit_H = pd.concat([train_visit_H, valid_visit_H], axis=0)

In [8]:
visit_H.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51596 entries, 0 to 5586
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   VISIT_AREA_ID        51596 non-null  int64  
 1   TRAVEL_ID            51596 non-null  object 
 2   VISIT_ORDER          51596 non-null  int64  
 3   VISIT_AREA_NM        51596 non-null  object 
 4   VISIT_START_YMD      51596 non-null  object 
 5   VISIT_END_YMD        51596 non-null  object 
 6   ROAD_NM_ADDR         43659 non-null  object 
 7   LOTNO_ADDR           44603 non-null  object 
 8   X_COORD              42817 non-null  float64
 9   Y_COORD              42817 non-null  float64
 10  ROAD_NM_CD           1503 non-null   float64
 11  LOTNO_CD             1503 non-null   float64
 12  POI_ID               42952 non-null  object 
 13  POI_NM               42952 non-null  object 
 14  RESIDENCE_TIME_MIN   48715 non-null  float64
 15  VISIT_AREA_TYPE_CD   51596 non-null  int64

### VISIT_AREA_TYPE_CD
- "21",집,
- "22",친구/친지집,
- "23",사무실,
- "24",숙소,
- "13",체험 활동 관광지,
- "1",자연관광지,
- "2","역사/유적/종교 시설(문화재, 박물관, 촬영지, 절 등)",
- "3","문화 시설(공연장, 영화관, 전시관 등)",
- "4","상업지구(거리, 시장, 쇼핑시설)",
- "5","레저/스포츠 관련 시설(스키, 카트, 수상레저)",
- "6","테마시설(놀이공원, 워터파크)",
- "7","산책로, 둘레길 등",
- "8",지역 축제/행사,
- "9","역, 터미널, 고속도로 휴게소",
- "10",상점
- "11",식당/카페
- "12",기타,

In [3]:
# ---------------------------------------------------------
# 데이터 고르기, 일단 명소가 아닌 곳만 제외하고 모두 선택,
# 나중에 테마별로 골라도 좋을 듯
# ---------------------------------------------------------

id_list=['H']

visit_data_list=[visit_H]

for id,visit_area_info in zip(id_list,visit_data_list):
  # 관광지 선택
  visit_info = visit_area_info[ (visit_area_info['VISIT_AREA_TYPE_CD'] == 1) |
   (visit_area_info['VISIT_AREA_TYPE_CD'] == 2) |(visit_area_info['VISIT_AREA_TYPE_CD'] == 3) | (visit_area_info['VISIT_AREA_TYPE_CD'] == 4) |
    (visit_area_info['VISIT_AREA_TYPE_CD'] == 5) | (visit_area_info['VISIT_AREA_TYPE_CD'] == 6) |(visit_area_info['VISIT_AREA_TYPE_CD'] == 7) |
     (visit_area_info['VISIT_AREA_TYPE_CD'] == 8)]
  visit_info = visit_info.groupby('VISIT_AREA_NM').filter(lambda x: len(x) > 1)
  visit_info=visit_info.reset_index(drop = True)

In [4]:
visit_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11181 entries, 0 to 11180
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   VISIT_AREA_ID        11181 non-null  int64  
 1   TRAVEL_ID            11181 non-null  object 
 2   VISIT_ORDER          11181 non-null  int64  
 3   VISIT_AREA_NM        11181 non-null  object 
 4   VISIT_START_YMD      11181 non-null  object 
 5   VISIT_END_YMD        11181 non-null  object 
 6   ROAD_NM_ADDR         6686 non-null   object 
 7   LOTNO_ADDR           11000 non-null  object 
 8   X_COORD              10820 non-null  float64
 9   Y_COORD              10820 non-null  float64
 10  ROAD_NM_CD           0 non-null      float64
 11  LOTNO_CD             0 non-null      float64
 12  POI_ID               10873 non-null  object 
 13  POI_NM               10873 non-null  object 
 14  RESIDENCE_TIME_MIN   11181 non-null  float64
 15  VISIT_AREA_TYPE_CD   11181 non-null 

In [5]:
visit_info.isnull().sum()

VISIT_AREA_ID              0
TRAVEL_ID                  0
VISIT_ORDER                0
VISIT_AREA_NM              0
VISIT_START_YMD            0
VISIT_END_YMD              0
ROAD_NM_ADDR            4495
LOTNO_ADDR               181
X_COORD                  361
Y_COORD                  361
ROAD_NM_CD             11181
LOTNO_CD               11181
POI_ID                   308
POI_NM                   308
RESIDENCE_TIME_MIN         0
VISIT_AREA_TYPE_CD         0
REVISIT_YN                 0
VISIT_CHC_REASON_CD        0
LODGING_TYPE_CD        11156
DGSTFN                     0
REVISIT_INTENTION          0
RCMDTN_INTENTION           0
SGG_CD                 11002
dtype: int64

In [12]:
print(visit_info[:20])

    VISIT_AREA_ID  TRAVEL_ID  VISIT_ORDER   VISIT_AREA_NM VISIT_START_YMD  \
0      2306100004  e_e000305            6          실미해수욕장      2023-06-10   
1      2306170002  e_e000335            2     타임 테라스 A 블록      2023-06-17   
2      2306170006  e_e000335            6       스타필드 코엑스몰      2023-06-17   
3      2306170008  e_e000335            8         잠실종합운동장      2023-06-17   
4      2305270005  e_e000401            5     시흥 프리미엄 아울렛      2023-05-27   
5      2306040004  e_e000437            3            선녀바위      2023-06-04   
6      2306040006  e_e000437            5        을왕리 해수욕장      2023-06-04   
7      2306030005  e_e000443            5             장봉도      2023-06-03   
8      2306030002  e_e000506            5             전등사      2023-06-03   
9      2306030003  e_e000506            6             초지진      2023-06-03   
10     2306030006  e_e000506            8     대명항 수산물 직판장      2023-06-03   
11     2306030007  e_e000506            9          김포함상공원      2023-06-03   

In [6]:
# ---------------------------------------------------------
# rating 계산
# ---------------------------------------------------------
visit_final_H=visit_info

visit_final_H['ratings'] = visit_final_H[['DGSTFN', 'REVISIT_INTENTION', 'RCMDTN_INTENTION']].mean(axis=1)

visit_final_H['TRAVELER_ID'] = visit_final_H['TRAVEL_ID'].str.split('_').str[1]


In [7]:
# ---------------------------------------------------------
# 시도 구분, 같은 장소 통합
# ---------------------------------------------------------
visit_final_H['SIDO'] = visit_final_H['LOTNO_ADDR'].str.split().str[0]

dfh=visit_final_H
# Group by 'FIRST_WORD' and find the most frequent 'VISIT_AREA_NM' for each group
most_frequent_visits = dfh.groupby('LOTNO_ADDR')['VISIT_AREA_NM'].agg(lambda x: x.mode().iloc[0]).reset_index()


In [15]:
print(most_frequent_visits[:30])

                LOTNO_ADDR   VISIT_AREA_NM
0   강원 강릉시 강동면 정동진리 524-25         하슬라아트월드
1         강원 강릉시 강문동 182-1            강문해변
2           강원 강릉시 강문동 산 1          경포해수욕장
3        강원 강릉시 견소동 286-10           안목 해변
4     강원 강릉시 사천면 방동리 282-1            사천해변
5            강원 강릉시 성남동 50         강릉 중앙시장
6          강원 강릉시 주문진읍 교항리         주문진 방사제
7     강원 강릉시 주문진읍 향호리 8-35           주문진해변
8         강원 강릉시 죽헌동 177-4             오죽헌
9           강원 강릉시 초당동 505      아르떼 뮤지엄 강릉
10         강원 고성군 죽왕면 문암진리             능파대
11          강원 고성군 토성면 봉포리           봉포 해변
12         강원 고성군 토성면 아야진리         아야진해수욕장
13       강원 동해시 묵호진동 13-61       도째비골해랑전망대
14       강원 동해시 묵호진동 2-109       도째비골스카이밸리
15       강원 동해시 묵호진동 2-294            논골담길
16        강원 속초시 교동 658-53          민우당 서림
17        강원 속초시 대포동 585-5      외옹치 바다 향기로
18        강원 속초시 동명동 1-185             영금정
19          강원 속초시 설악동 170             신흥사
20          강원 속초시 영랑동 1-9          등대해수욕장
21         강원 속초시 장사동 24-9          설악워터피아
22     강원 속

In [8]:
dfh = dfh.merge(most_frequent_visits, on='LOTNO_ADDR', how='left', suffixes=('', '_most_frequent'))

# Update 'VISIT_AREA_NM' with the most frequent values
dfh['VISIT_AREA_NM'] = dfh['VISIT_AREA_NM_most_frequent'].fillna(dfh['VISIT_AREA_NM'])


In [9]:
dfh.drop(columns=['VISIT_AREA_NM_most_frequent'], inplace=True)

In [11]:


dfh[['TRAVELER_ID', 'VISIT_AREA_NM','ratings','SIDO', 'VISIT_AREA_TYPE_CD','ROAD_NM_ADDR','LOTNO_ADDR','X_COORD','Y_COORD']]

,TRAVELER_ID,VISIT_AREA_NM,ratings,SIDO,VISIT_AREA_TYPE_CD,ROAD_NM_ADDR,LOTNO_ADDR,X_COORD,Y_COORD
0,e000305,실미해수욕장,1.000000,인천,1,인천 중구 큰무리로 108,인천 중구 무의동 768-29,NaN,NaN
1,e000335,타임 테라스 A 블록,4.666667,경기,4,경기 화성시 동탄중앙로 220,경기 화성시 반송동 96,127.069243,37.204920
2,e000335,스타필드 코엑스몰,5.000000,서울,4,서울 강남구 영동대로 513,서울 강남구 삼성동 159-9,127.059777,37.511318
3,e000335,잠실종합운동장,5.000000,서울,3,서울 송파구 올림픽로 25,서울 송파구 잠실동 10-1,127.073025,37.514591
4,e000401,시흥 프리미엄 아울렛,2.666667,경기,4,경기 시흥시 서해안로 699,경기 시흥시 배곧동 36,126.737081,37.379781
...,...,...,...,...,...,...,...,...,...
11176,h006731,제지기 오름,3.000000,NaN,1,NaN,NaN,126.622405,33.254246
11177,h006731,쇠소깍,5.000000,NaN,1,NaN,NaN,126.622405,33.254246
11178,h006753,성이시돌목장,5.000000,제주특별자치도,7,제주특별자치도 제주시 한림읍 금악동길 35,제주특별자치도 제주시 한림읍 금악리 142,126.327321,33.347770
11179,h006753,금오름,5.000000,제주특별자치도,1,NaN,제주특별자치도 제주시 한림읍 금악리 산 1-1,126.305839,33.354572


In [ ]:
# ---------------------------------------------------------
# 시도가 빈 곳은 같은 장소에 채워진 시도로 메움
# 나머지 빈 곳은 드랍
# ---------------------------------------------------------


#df1 = dfh.rename(columns={'TRAVELER_ID': 'userID','VISIT_AREA_NM': 'itemID','ratings': 'rating'})

item_info=dfh[['TRAVELER_ID','VISIT_AREA_NM','ratings','SIDO','VISIT_AREA_TYPE_CD']]
item_info['SIDO'] = item_info.groupby('VISIT_AREA_NM')['SIDO'].transform(
    lambda x: x.ffill().bfill()
)
item_info.dropna(inplace=True)
item_info.to_csv(save_path / "rating_item_info_all_sido.csv")


C:\Users\User\AppData\Local\Temp\ipykernel_4600\3912354621.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lambda x: x.ffill().bfill()
C:\Users\User\AppData\Local\Temp\ipykernel_4600\3912354621.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  item_info['SIDO'] = item_info.groupby('VISIT_AREA_NM')['SIDO'].transform(
C:\Users\User\AppData\Local\Temp\ipykernel_4600\3912354621.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.py

In [27]:
print(item_info.isnull().sum())
print(len(item_info))

TRAVELER_ID           0
VISIT_AREA_NM         0
ratings               0
SIDO                  0
VISIT_AREA_TYPE_CD    0
dtype: int64
11115


In [28]:
item_info.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11115 entries, 0 to 11180
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   TRAVELER_ID         11115 non-null  object 
 1   VISIT_AREA_NM       11115 non-null  object 
 2   ratings             11115 non-null  float64
 3   SIDO                11115 non-null  object 
 4   VISIT_AREA_TYPE_CD  11115 non-null  int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 521.0+ KB


# 여행객 정보

In [ ]:
# ---------------------------------------------------------
# 데이터 Load, Concat
# ---------------------------------------------------------

tn_traveller_master_path = tn_base_path / "tn_traveller_master_여행객 Master_H.csv"
vn_traveller_master_path = vn_base_path / "tn_traveller_master_여행객 Master_H.csv"

train_user_info_H = pd.read_csv(tn_traveller_master_path)
valid_user_info_H = pd.read_csv(vn_traveller_master_path)
user_info_H = pd.concat([train_user_info_H, valid_user_info_H], axis=0)

In [31]:
user_info_H.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2880 entries, 0 to 319
Data columns (total 36 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   TRAVELER_ID                2880 non-null   object 
 1   RESIDENCE_SGG_CD           2880 non-null   int64  
 2   GENDER                     2880 non-null   object 
 3   AGE_GRP                    2880 non-null   int64  
 4   EDU_NM                     2880 non-null   int64  
 5   EDU_FNSH_SE                2880 non-null   int64  
 6   MARR_STTS                  2880 non-null   int64  
 7   FAMILY_MEMB                2880 non-null   int64  
 8   JOB_NM                     2880 non-null   int64  
 9   JOB_ETC                    0 non-null      float64
 10  INCOME                     2880 non-null   int64  
 11  HOUSE_INCOME               2190 non-null   float64
 12  TRAVEL_TERM                2880 non-null   int64  
 13  TRAVEL_NUM                 2880 non-null   int64  
 14

In [32]:
user_info_H.isnull().sum()

TRAVELER_ID                     0
RESIDENCE_SGG_CD                0
GENDER                          0
AGE_GRP                         0
EDU_NM                          0
EDU_FNSH_SE                     0
MARR_STTS                       0
FAMILY_MEMB                     0
JOB_NM                          0
JOB_ETC                      2880
INCOME                          0
HOUSE_INCOME                  690
TRAVEL_TERM                     0
TRAVEL_NUM                      0
TRAVEL_LIKE_SIDO_1              0
TRAVEL_LIKE_SGG_1               0
TRAVEL_LIKE_SIDO_2              0
TRAVEL_LIKE_SGG_2               0
TRAVEL_LIKE_SIDO_3              0
TRAVEL_LIKE_SGG_3               0
TRAVEL_STYL_1                   0
TRAVEL_STYL_2                   0
TRAVEL_STYL_3                   0
TRAVEL_STYL_4                   0
TRAVEL_STYL_5                   0
TRAVEL_STYL_6                   0
TRAVEL_STYL_7                   0
TRAVEL_STYL_8                   0
TRAVEL_STATUS_RESIDENCE         0
TRAVEL_STATUS_

In [33]:
user_info_H.columns

Index(['TRAVELER_ID', 'RESIDENCE_SGG_CD', 'GENDER', 'AGE_GRP', 'EDU_NM',
       'EDU_FNSH_SE', 'MARR_STTS', 'FAMILY_MEMB', 'JOB_NM', 'JOB_ETC',
       'INCOME', 'HOUSE_INCOME', 'TRAVEL_TERM', 'TRAVEL_NUM',
       'TRAVEL_LIKE_SIDO_1', 'TRAVEL_LIKE_SGG_1', 'TRAVEL_LIKE_SIDO_2',
       'TRAVEL_LIKE_SGG_2', 'TRAVEL_LIKE_SIDO_3', 'TRAVEL_LIKE_SGG_3',
       'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4',
       'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8',
       'TRAVEL_STATUS_RESIDENCE', 'TRAVEL_STATUS_DESTINATION',
       'TRAVEL_STATUS_ACCOMPANY', 'TRAVEL_STATUS_YMD', 'TRAVEL_MOTIVE_1',
       'TRAVEL_MOTIVE_2', 'TRAVEL_MOTIVE_3', 'TRAVEL_COMPANIONS_NUM'],
      dtype='object')

In [ ]:
# ---------------------------------------------------------
# 데이터 mapping 겸 description
# ---------------------------------------------------------

map_dict = {
    'TRAVELER_ID':'여행객ID',
    'GENDER':'성별',
    'AGE_GRP':'연령대',
    'MARR_STTS':'혼인상태', # MAR
    'JOB_NM':'직업', # JOB
    'INCOME':'본인소득', # INC
    'TRAVEL_NUM':'여행빈도', # 연간 여행 빈도
    'TRAVEL_STYL_1':'여행스타일', # TSY
    'TRAVEL_STATUS_RESIDENCE':'거주지', #
    'TRAVEL_STATUS_DESTINATION':'목적지', #
    'TRAVEL_STATUS_ACCOMPANY':'동반현황',
    'TRAVEL_STATUS_YMD':'여행일자',
    'TRAVEL_MOTIVE_1':'여행동기', # TMT
    'TRAVEL_COMPANIONS_NUM':'여행동반자수'
}

In [35]:
map_dict.keys()

dict_keys(['TRAVELER_ID', 'GENDER', 'AGE_GRP', 'MARR_STTS', 'JOB_NM', 'INCOME', 'TRAVEL_NUM', 'TRAVEL_STYL_1', 'TRAVEL_STATUS_RESIDENCE', 'TRAVEL_STATUS_DESTINATION', 'TRAVEL_STATUS_ACCOMPANY', 'TRAVEL_STATUS_YMD', 'TRAVEL_MOTIVE_1', 'TRAVEL_COMPANIONS_NUM'])

In [ ]:
# ---------------------------------------------------------
# 데이터 column filtering
# ---------------------------------------------------------
user_info = user_info_H[list(map_dict.keys())]

In [37]:
print(len(user_info))
print(len(set(user_info.TRAVELER_ID.to_list())))

2880
2880


In [38]:
user_info.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2880 entries, 0 to 319
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   TRAVELER_ID                2880 non-null   object
 1   GENDER                     2880 non-null   object
 2   AGE_GRP                    2880 non-null   int64 
 3   MARR_STTS                  2880 non-null   int64 
 4   JOB_NM                     2880 non-null   int64 
 5   INCOME                     2880 non-null   int64 
 6   TRAVEL_NUM                 2880 non-null   int64 
 7   TRAVEL_STYL_1              2880 non-null   int64 
 8   TRAVEL_STATUS_RESIDENCE    2880 non-null   object
 9   TRAVEL_STATUS_DESTINATION  2880 non-null   object
 10  TRAVEL_STATUS_ACCOMPANY    2880 non-null   object
 11  TRAVEL_STATUS_YMD          2880 non-null   object
 12  TRAVEL_MOTIVE_1            2880 non-null   int64 
 13  TRAVEL_COMPANIONS_NUM      2880 non-null   int64 
dtypes: int64(8), o

In [39]:
user_info[:20]

,TRAVELER_ID,GENDER,AGE_GRP,MARR_STTS,JOB_NM,INCOME,TRAVEL_NUM,TRAVEL_STYL_1,TRAVEL_STATUS_RESIDENCE,TRAVEL_STATUS_DESTINATION,TRAVEL_STATUS_ACCOMPANY,TRAVEL_STATUS_YMD,TRAVEL_MOTIVE_1,TRAVEL_COMPANIONS_NUM
0,g003414,남,20,1,3,1,2,3,대전광역시,전남,2인 여행(가족 외),2023-07-08~2023-07-09,1,1
1,h006468,남,40,2,1,8,2,1,제주특별자치도,제주,자녀 동반 여행,2023-09-16~2023-09-16,8,2
2,h003272,여,50,1,2,4,1,3,제주특별자치도,제주,나홀로 여행,2023-08-27~2023-08-27,6,0
3,h006175,여,20,1,2,6,2,4,대구광역시,제주,나홀로 여행,2023-09-06~2023-09-07,1,0
4,e000899,여,30,1,2,5,1,3,서울특별시,경기,부모 동반 여행,2023-05-27~2023-05-29,2,2
5,h002865,여,30,1,12,1,1,3,부산광역시,제주,2인 여행(가족 외),2023-09-02~2023-09-04,3,1
6,e006309,남,30,1,13,2,2,3,서울특별시,도서 지역,나홀로 여행,2023-08-03~2023-08-05,7,0
7,h004106,여,20,1,3,3,7,1,광주광역시,제주,나홀로 여행,2023-08-17~2023-08-21,4,0
8,h000092,여,20,1,13,6,4,1,충청남도,제주,나홀로 여행,2023-05-23~2023-05-26,1,0
9,h000468,여,30,1,13,4,3,6,경기도,제주,2인 여행(가족 외),2023-06-08~2023-06-10,2,1


In [ ]:
# ---------------------------------------------------------
# 데이터 전처리
# ---------------------------------------------------------

# split해서 컬럼 분리
user_info[['start_date', 'end_date']] = user_info['TRAVEL_STATUS_YMD'].str.split('~', expand=True)

# datetime 변환
user_info['start_date'] = pd.to_datetime(user_info['start_date'])
user_info['end_date'] = pd.to_datetime(user_info['end_date'])

# month 추출 (시작일 기준)
user_info['MONTH'] = user_info['start_date'].dt.month

# season 생성
def get_season(m):
    if m in [12, 1, 2]:
        return 'winter'
    elif m in [3, 4, 5]:
        return 'spring'
    elif m in [6, 7, 8]:
        return 'summer'
    else:
        return 'autumn'

user_info['SEASON'] = user_info['MONTH'].apply(get_season)

# 여행 며칠 다녀왔는지
user_info['HOW_LONG'] = (user_info['end_date'] - user_info['start_date']).dt.days + 1


C:\Users\User\AppData\Local\Temp\ipykernel_4600\1270321213.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_info[['start_date', 'end_date']] = user_info['TRAVEL_STATUS_YMD'].str.split('~', expand=True)
C:\Users\User\AppData\Local\Temp\ipykernel_4600\1270321213.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_info[['start_date', 'end_date']] = user_info['TRAVEL_STATUS_YMD'].str.split('~', expand=True)
C:\Users\User\AppData\Local\Temp\ipykernel_4600\1270321213.py:5: SettingWithCopyWarning: 
A 

In [ ]:
# 다 쓴 컬럼 드랍
user_info = user_info.drop(['TRAVEL_STATUS_YMD', 'start_date', 'end_date'], axis=1)

In [42]:
user_info.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2880 entries, 0 to 319
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   TRAVELER_ID                2880 non-null   object
 1   GENDER                     2880 non-null   object
 2   AGE_GRP                    2880 non-null   int64 
 3   MARR_STTS                  2880 non-null   int64 
 4   JOB_NM                     2880 non-null   int64 
 5   INCOME                     2880 non-null   int64 
 6   TRAVEL_NUM                 2880 non-null   int64 
 7   TRAVEL_STYL_1              2880 non-null   int64 
 8   TRAVEL_STATUS_RESIDENCE    2880 non-null   object
 9   TRAVEL_STATUS_DESTINATION  2880 non-null   object
 10  TRAVEL_STATUS_ACCOMPANY    2880 non-null   object
 11  TRAVEL_MOTIVE_1            2880 non-null   int64 
 12  TRAVEL_COMPANIONS_NUM      2880 non-null   int64 
 13  MONTH                      2880 non-null   int32 
 14  SEASON        

In [43]:
user_info[:20]

,TRAVELER_ID,GENDER,AGE_GRP,MARR_STTS,JOB_NM,INCOME,TRAVEL_NUM,TRAVEL_STYL_1,TRAVEL_STATUS_RESIDENCE,TRAVEL_STATUS_DESTINATION,TRAVEL_STATUS_ACCOMPANY,TRAVEL_MOTIVE_1,TRAVEL_COMPANIONS_NUM,MONTH,SEASON,HOW_LONG
0,g003414,남,20,1,3,1,2,3,대전광역시,전남,2인 여행(가족 외),1,1,7,summer,2
1,h006468,남,40,2,1,8,2,1,제주특별자치도,제주,자녀 동반 여행,8,2,9,autumn,1
2,h003272,여,50,1,2,4,1,3,제주특별자치도,제주,나홀로 여행,6,0,8,summer,1
3,h006175,여,20,1,2,6,2,4,대구광역시,제주,나홀로 여행,1,0,9,autumn,2
4,e000899,여,30,1,2,5,1,3,서울특별시,경기,부모 동반 여행,2,2,5,spring,3
5,h002865,여,30,1,12,1,1,3,부산광역시,제주,2인 여행(가족 외),3,1,9,autumn,3
6,e006309,남,30,1,13,2,2,3,서울특별시,도서 지역,나홀로 여행,7,0,8,summer,3
7,h004106,여,20,1,3,3,7,1,광주광역시,제주,나홀로 여행,4,0,8,summer,5
8,h000092,여,20,1,13,6,4,1,충청남도,제주,나홀로 여행,1,0,5,spring,4
9,h000468,여,30,1,13,4,3,6,경기도,제주,2인 여행(가족 외),2,1,6,summer,3


In [ ]:
# ---------------------------------------------------------
# 데이터 저장
# ---------------------------------------------------------

user_info.to_csv(save_path / "rating_user_info_all.csv")

## data join

In [12]:
import pandas as pd
import numpy as np
from pathlib import Path
from scipy.sparse import coo_matrix

BASE_DIR = Path().resolve().parent.parent.parent
# base_path =  BASE_DIR / "data" / "train" / "TL_csv"
base_path = BASE_DIR / "AI 모델" / "1.모델소스코드" / "1.여행로그 장소 추천 고도화" / "data"
save_path = BASE_DIR / "AI 모델" / "1.모델소스코드" / "1.여행로그 장소 추천 고도화" / "model"

In [13]:
import pickle
import json
import pandas as pd
from scipy.sparse import load_npz


# 1) LightFM 모델 로드
with open(save_path / "lightfm_model.pkl", "rb") as f:
    model = pickle.load(f)

# 2) item_features_matrix 로드
item_features_matrix = load_npz(save_path / "item_features_matrix.npz")

# 3) item 메타데이터 로드
item_meta = pd.read_csv(save_path / "item_meta.csv")

c:\Users\User\Desktop\xodls\through_pjt_ai\venv_lightfm_real\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [25]:
dfh_info = dfh[['VISIT_AREA_NM','ROAD_NM_ADDR','LOTNO_ADDR','X_COORD','Y_COORD']].drop_duplicates(subset=['VISIT_AREA_NM'])

In [26]:
item_meta2 = item_meta.merge(dfh_info, on='VISIT_AREA_NM', how='left', suffixes=('', '_'))

In [27]:
item_meta2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1257 entries, 0 to 1256
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   item_id             1257 non-null   int64  
 1   VISIT_AREA_NM       1257 non-null   object 
 2   SIDO                1257 non-null   object 
 3   VISIT_AREA_TYPE_CD  1257 non-null   int64  
 4   ratings             1257 non-null   float64
 5   ROAD_NM_ADDR        836 non-null    object 
 6   LOTNO_ADDR          1240 non-null   object 
 7   X_COORD             1223 non-null   float64
 8   Y_COORD             1223 non-null   float64
dtypes: float64(3), int64(2), object(4)
memory usage: 98.2+ KB


In [24]:
item_meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1257 entries, 0 to 1256
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   item_id             1257 non-null   int64  
 1   VISIT_AREA_NM       1257 non-null   object 
 2   SIDO                1257 non-null   object 
 3   VISIT_AREA_TYPE_CD  1257 non-null   int64  
 4   ratings             1257 non-null   float64
dtypes: float64(1), int64(2), object(2)
memory usage: 49.2+ KB


In [28]:
item_meta2.to_csv(save_path / "item_meta.csv")